In [1]:
import pennylane as qml
import pennylane.numpy as np
import torch
from torch.nn import MSELoss, CrossEntropyLoss
from torch.optim import Adam, Adagrad
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split


import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
EPOCHS = 1

In [3]:
from QCNN_circuit_only import QCNN
from ansatz import ConvCirc1, PoolingCirc

qcnn = QCNN(n_qubits=8, conv_ansatz=ConvCirc1(), pooling_ansatz=PoolingCirc(), stride=1)

dev = qml.device('default.qubit', wires=qcnn.total_qubit)

@qml.qnode(dev, interface='torch')
def circuit(params, data):
    qcnn.construct_circuit(params, data)
    return qml.expval(qml.PauliZ(0))


In [4]:
from pandas import read_csv, concat

class MyDataset(Dataset):
    def __init__(self, file_name):
        self.data = read_csv(file_name)

    def filter(self, labels):
        data_list  = []
        for label in labels:
            data_list.append(self.data[self.data["label"]==label])
        self.data = concat(data_list)

    def __getitem__(self, index):
        label = self.data.iloc[index]['label']
        image = self.data.iloc[index][1:]
        return torch.tensor(image), torch.tensor(label)

    def __len__(self):
        return len(self.data)

In [5]:

# TODO: use postprocessed data instead
# ====================================
#transform = transforms.Compose(
#    [transforms.ToTensor(),
#    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & test, download if necessary
#training_set = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True)
#training_set, validation_set = random_split(training_set, [50000, 10000])
#test_set = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True)
training_set = MyDataset('dataset_pca.csv') 
training_set.filter([0, 1])#torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True)
training_set, validation_set = random_split(training_set, [1000, len(training_set)-1000])
validation_set, test_set = random_split(validation_set, [1000, len(validation_set)-1000])

# Create data loaders for our datasets; shuffle for training, not for test
training_loader = DataLoader(training_set, batch_size=25, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=25, shuffle=False)
test_loader = DataLoader(test_set, batch_size=25, shuffle=False)

# ====================================

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('validation set has {} instances'.format(len(validation_set)))
print('test set has {} instances'.format(len(test_set)))

Training set has 1000 instances
validation set has 1000 instances
test set has 115 instances


In [6]:
loss_fn = MSELoss() # TODO: choose loss function
params = torch.rand(qcnn.Calculate_Param_Num(), requires_grad=True) # TODO: choose params length

optimizer = Adam([params], lr=0.01) # TODO: choose optimizer


In [14]:
from tqdm.notebook import tqdm


def estimated_label(params, data):
    return circuit(params, data) # TODO: customize your extimated label

writer = SummaryWriter(log_dir='./runs')

for epoch in range(EPOCHS):
    avg_loss = 0.0
    for i, data in tqdm(enumerate(training_loader), total=len(training_loader), desc=f"{epoch}"):
        inputs, labels = data
        optimizer.zero_grad()
        loss = torch.tensor(0, dtype=torch.float64)
        for input, label in zip(inputs, labels):
            print(input.shape)
            output = estimated_label(params, inputs.to(torch.float64))
            loss+=loss_fn(output, label.to(torch.float64))/len(labels)
        loss.backward()       
        optimizer.step()
        avg_loss += loss.item()
        
    avg_loss = avg_loss/len(training_loader)
    
    with torch.no_grad():
        avg_vloss = 0.0
        for i, vdata in enumerate(test_loader):
            vloss = torch.tensor(0, dtype=torch.float64)
            vinputs, vlabels = vdata
            for vinput, vlabel in zip(vinputs, vlabels):
                voutput = estimated_label(params, vinput.to(torch.float64))
                vloss += loss_fn(voutput, vlabel.to(torch.float64))/len(vlabels)
            avg_vloss += vloss
        avg_vloss = avg_vloss/len(test_loader)
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and test
    writer.add_scalars('Training vs. test Loss',
                    { 'Training' : avg_loss, 'test' : avg_vloss },
                    epoch)
    writer.flush()
        # print('{}: {}'.format(i, loss.item()))


0:   0%|          | 0/40 [00:00<?, ?it/s]

torch.Size([256])


ValueError: State vector must be of length 2**wires.

In [ ]:
next(iter(training_loader))

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/shaun/anaconda3/envs/qiskit/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/shaun/anaconda3/envs/qiskit/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MyDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/shaun/anaconda3/envs/qiskit/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/shaun/anaconda3/envs/qiskit/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MyDataset' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 40817) exited unexpectedly

In [11]:
next(iter(training_loader))[0].shape

torch.Size([25, 256])

In [12]:
qml.draw(circuit)(params, next(iter(training_loader))[0][0])

KeyError: tensor(0, requires_grad=True)

In [15]:
circuit(params, next(iter(training_loader))[0][0])

WireError: Did not find some of the wires (tensor(0, requires_grad=True),) on device with wires (0, 1, 2, 3, 4, 5, 6, 7).

In [ ]:
qcnn.total_qubit

8